# Data Load

In [1]:
import config
from pshmodule.utils import filemanager as fm

In [2]:
df = fm.load(config.origin_data)

extension : .pickle
Loaded 10317 records from /Volumes/GoogleDrive/내 드라이브/MemeProject/data/temp_for_doc2vec.pickle


In [3]:
df.head()

,content,pos
0,나 중간고사 반에서 1등했어,"[나, 중간고사, 반]"
1,중간고사 점수 내가 반에서 제일 잘 받음,"[중간고사, 점수, 내, 반, 제일, 받음]"
2,나 반에서 중간 성적 제일 좋아,"[나, 반, 중간, 성적, 제일, 좋아]"
3,우리 반에서 내가 시험 제일 잘 봤다,"[우리, 반, 내, 시험, 제일, 봤다]"
4,중간고사 반 1등 먹음,"[중간고사, 반, 먹음]"


# 

# word2vec

In [4]:
import swifter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### vocabulary

In [7]:
vocab_list = []
for sentence in df.pos_str:
    for s in sentence:
        vocab_list.append(s)

In [8]:
vocab = set(vocab_list)

In [9]:
vocab_size = len(vocab)

In [10]:
print(vocab_size)

1211


In [11]:
w2i = {w: i for i, w in enumerate(vocab)}
i2w = {i: w for i, w in enumerate(vocab)}

In [12]:
print(f"w2i : {w2i}")
print(f"i2w : {i2w}")

w2i : {'뛰': 0, '눔': 1, '랜': 2, '입': 3, '첨': 4, '연': 5, '컴': 6, '끔': 7, '엎': 8, '생': 9, '확': 10, '떡': 11, '힙': 12, '엌': 13, '딱': 14, '멘': 15, '드': 16, '칠': 17, '샵': 18, '산': 19, '빙': 20, '옷': 21, '뀄': 22, '펌': 23, '깥': 24, '층': 25, '빽': 26, '욀': 27, '엽': 28, '순': 29, '없': 30, '플': 31, '꼭': 32, '끄': 33, '터': 34, '뽀': 35, '눈': 36, '쩜': 37, '워': 38, '낀': 39, '넓': 40, '쩔': 41, '췄': 42, '내': 43, '닦': 44, '짰': 45, '젠': 46, '몰': 47, '톤': 48, '꾸': 49, '쿡': 50, '욤': 51, '얺': 52, '싫': 53, '툰': 54, '판': 55, '리': 56, '폰': 57, '훈': 58, '못': 59, '좔': 60, '흉': 61, '랑': 62, '멤': 63, '신': 64, '맑': 65, '찬': 66, '페': 67, '렐': 68, '건': 69, '셋': 70, '펠': 71, '암': 72, '왤': 73, '찔': 74, '릭': 75, '귤': 76, '호': 77, '쉬': 78, '습': 79, '음': 80, '시': 81, '씀': 82, '듦': 83, '흰': 84, '오': 85, '쇠': 86, '딸': 87, '식': 88, '벨': 89, '냠': 90, '뜯': 91, '뢰': 92, '롯': 93, '쁠': 94, '길': 95, '말': 96, '쇼': 97, '룩': 98, '뭣': 99, '커': 100, '춘': 101, '림': 102, '붓': 103, '먼': 104, '뉴': 105, '킹': 106, '측': 107, '환': 108, '쾌': 109, '션'

### dataset

In [23]:
# context window size is two
def create_cbow_dataset(text):
    data = []
    for i in range(2, len(text) - 2):
        context = [text[i - 2], text[i - 1],
                   text[i + 1], text[i + 2]]
        target = text[i]
        data.append((context, target))
    return data

In [24]:
def create_skipgram_dataset(text):
    import random
    data = []
    for i in range(2, len(text) - 2):
        data.append((text[i], text[i-2], 1))
        data.append((text[i], text[i-1], 1))
        data.append((text[i], text[i+1], 1))
        data.append((text[i], text[i+2], 1))
        # negative sampling
        for _ in range(4):
            if random.random() < 0.5 or i >= len(text) - 3:
                rand_id = random.randint(0, i-1)
            else:
                rand_id = random.randint(i+3, len(text)-1)
            data.append((text[i], text[rand_id], 0))
    return data

In [25]:
df['cbow'] = df.pos.swifter.apply(create_cbow_dataset)
df['skipgram'] = df.pos.swifter.apply(create_skipgram_dataset)

Pandas Apply:   0%|          | 0/10317 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/10317 [00:00<?, ?it/s]

In [26]:
df.head()

,content,pos,cbow,skipgram
0,나 중간고사 반에서 1등했어,"[나, 중간고사, 반]",[],[]
1,중간고사 점수 내가 반에서 제일 잘 받음,"[중간고사, 점수, 내, 반, 제일, 받음]","[([중간고사, 점수, 반, 제일], 내), ([점수, 내, 제일, 받음], 반)]","[(내, 중간고사, 1), (내, 점수, 1), (내, 반, 1), (내, 제일, ..."
2,나 반에서 중간 성적 제일 좋아,"[나, 반, 중간, 성적, 제일, 좋아]","[([나, 반, 성적, 제일], 중간), ([반, 중간, 제일, 좋아], 성적)]","[(중간, 나, 1), (중간, 반, 1), (중간, 성적, 1), (중간, 제일,..."
3,우리 반에서 내가 시험 제일 잘 봤다,"[우리, 반, 내, 시험, 제일, 봤다]","[([우리, 반, 시험, 제일], 내), ([반, 내, 제일, 봤다], 시험)]","[(내, 우리, 1), (내, 반, 1), (내, 시험, 1), (내, 제일, 1)..."
4,중간고사 반 1등 먹음,"[중간고사, 반, 먹음]",[],[]


### model

In [11]:
class SkipGram(nn.Module):
    def __init__(self, vocab_size, embd_size):
        super(SkipGram, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embd_size)
    
    def forward(self, focus, context):
        embed_focus = self.embeddings(focus).view((1, -1))
        embed_ctx = self.embeddings(context).view((1, -1))
        score = torch.mm(embed_focus, torch.t(embed_ctx))
        log_probs = F.logsigmoid(score)
    
        return log_probs

In [ ]:
def train_skipgram():
    losses = []
    loss_fn = nn.MSELoss()
    model = SkipGram(vocab_size, embd_size)
    print(model)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    for epoch in range(n_epoch):
        total_loss = .0
        for in_w, out_w, target in skipgram_train:
            in_w_var = Variable(torch.LongTensor([w2i[in_w]]))
            out_w_var = Variable(torch.LongTensor([w2i[out_w]]))
            
            model.zero_grad()
            log_probs = model(in_w_var, out_w_var)
            loss = loss_fn(log_probs[0], Variable(torch.Tensor([target])))
            
            loss.backward()
            optimizer.step()

            total_loss += loss.data[0]
        losses.append(total_loss)
    return model, losses

In [ ]:
embd_size = 100
learning_rate = 0.001
n_epoch = 30

In [ ]:
sg_model, sg_losses = train_skipgram()